# Module 0.2: 梯度與微分

這個 notebook 會教你理解梯度的意義，並學會手推簡單函數的梯度。

## 學習目標

1. 理解梯度的幾何意義
2. 對純量函數求向量梯度
3. 鏈式法則 (Chain Rule)
4. 用數值微分驗證解析梯度

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

---
## Part 1: 什麼是梯度？

### 回顧：一維的導數

對於一維函數 $f(x)$，導數 $f'(x)$ 告訴我們：
- 函數在 $x$ 點的**變化率**
- 如果 $f'(x) > 0$，函數在增加
- 如果 $f'(x) < 0$，函數在減少

### 多維的梯度

對於多維函數 $f(\mathbf{x})$ 其中 $\mathbf{x} = [x_1, x_2, ..., x_n]$：

$$\nabla f(\mathbf{x}) = \begin{bmatrix} \frac{\partial f}{\partial x_1} \\ \frac{\partial f}{\partial x_2} \\ \vdots \\ \frac{\partial f}{\partial x_n} \end{bmatrix}$$

梯度是一個**向量**，它的每個元素是 $f$ 對每個變數的偏導數。

### 梯度的幾何意義（超重要！）

**梯度指向函數增長最快的方向**

這就是為什麼梯度下降要往 $-\nabla f$ 方向走——因為那是函數**減少**最快的方向！

In [ ]:
# 視覺化 2D 函數的梯度
# f(x, y) = x² + y²（一個碗狀的函數）

def f(x, y):
    return x**2 + y**2

def grad_f(x, y):
    """梯度 = [2x, 2y]"""
    return np.array([2*x, 2*y])

# 創建網格
x = np.linspace(-2, 2, 20)
y = np.linspace(-2, 2, 20)
X, Y = np.meshgrid(x, y)
Z = f(X, Y)

# 畫等高線和梯度向量
plt.figure(figsize=(10, 8))
plt.contour(X, Y, Z, levels=15, cmap='viridis')
plt.colorbar(label='f(x, y)')

# 在幾個點畫梯度向量
points = [(-1.5, -1), (0.5, 1.5), (1, -0.5), (-0.5, 0.5)]
for px, py in points:
    gx, gy = grad_f(px, py)
    # 縮放梯度長度以便顯示
    scale = 0.2
    plt.arrow(px, py, gx*scale, gy*scale, 
              head_width=0.1, head_length=0.05, fc='red', ec='red')
    plt.plot(px, py, 'ro', markersize=8)

plt.xlabel('x')
plt.ylabel('y')
plt.title('f(x,y) = x² + y² 的等高線和梯度\n紅色箭頭指向函數增長最快的方向')
plt.axis('equal')
plt.grid(True, alpha=0.3)
plt.show()

---
## Part 2: 常見函數的梯度

### 例 1：線性函數

$$f(\mathbf{x}) = \mathbf{w}^T \mathbf{x} = w_1 x_1 + w_2 x_2 + \cdots + w_n x_n$$

$$\nabla_\mathbf{x} f = \mathbf{w}$$

### 例 2：L2 範數平方

$$f(\mathbf{x}) = ||\mathbf{x}||_2^2 = x_1^2 + x_2^2 + \cdots + x_n^2$$

$$\nabla f = 2\mathbf{x}$$

### 例 3：二次型（Quadratic Form）

$$f(\mathbf{x}) = \mathbf{x}^T A \mathbf{x}$$

其中 A 是對稱矩陣：

$$\nabla f = 2A\mathbf{x}$$

如果 A 不是對稱的：

$$\nabla f = (A + A^T)\mathbf{x}$$

### 練習 1: 實作這些梯度

**提示**：
- 線性函數的梯度就是係數向量
- L2 範數平方的梯度是 2x
- 二次型需要用矩陣運算

In [ ]:
def linear_function(x, w):
    """f(x) = w^T x"""
    return np.dot(w, x)

def grad_linear(x, w):
    """∇f = w"""
    # 解答：
    return w.copy()


def l2_squared(x):
    """f(x) = ||x||² = x₁² + x₂² + ..."""
    return np.sum(x ** 2)

def grad_l2_squared(x):
    """∇f = 2x"""
    # 解答：
    return 2 * x


def quadratic_form(x, A):
    """f(x) = x^T A x"""
    return x @ A @ x

def grad_quadratic_form(x, A):
    """∇f = (A + A^T) x"""
    # 解答：
    return (A + A.T) @ x

---
## Part 3: 數值微分（用來驗證你的梯度）

如何知道你手推的梯度對不對？用**數值微分**來驗證！

### 前向差分

$$\frac{\partial f}{\partial x_i} \approx \frac{f(x + \epsilon e_i) - f(x)}{\epsilon}$$

### 中央差分（更準確！）

$$\frac{\partial f}{\partial x_i} \approx \frac{f(x + \epsilon e_i) - f(x - \epsilon e_i)}{2\epsilon}$$

其中 $e_i$ 是第 $i$ 個維度的單位向量，$\epsilon$ 是很小的數（通常 $10^{-5}$）。

In [ ]:
def numerical_gradient(f, x, eps=1e-5):
    """
    用中央差分計算數值梯度
    
    Parameters
    ----------
    f : callable
        純量函數 f(x) -> float
    x : np.ndarray
        計算梯度的位置
    eps : float
        差分步長
    
    Returns
    -------
    grad : np.ndarray
        數值梯度
    """
    # 解答：
    grad = np.zeros_like(x, dtype=float)
    
    for i in range(len(x)):
        # 保存原值
        old_val = x[i]
        
        # f(x + eps)
        x[i] = old_val + eps
        f_plus = f(x)
        
        # f(x - eps)
        x[i] = old_val - eps
        f_minus = f(x)
        
        # 中央差分
        grad[i] = (f_plus - f_minus) / (2 * eps)
        
        # 恢復原值
        x[i] = old_val
    
    return grad


def gradient_check(analytic_grad, f, x, eps=1e-5):
    """
    比較解析梯度和數值梯度
    
    Returns
    -------
    relative_error : float
        相對誤差，應該 < 1e-5
    """
    numeric_grad = numerical_gradient(f, x.copy(), eps)
    
    # 計算相對誤差
    diff = np.linalg.norm(analytic_grad - numeric_grad)
    norm_sum = np.linalg.norm(analytic_grad) + np.linalg.norm(numeric_grad)
    
    if norm_sum == 0:
        relative_error = 0
    else:
        relative_error = diff / norm_sum
    
    print(f"Analytic gradient:  {analytic_grad}")
    print(f"Numerical gradient: {numeric_grad}")
    print(f"Relative error: {relative_error:.2e}")
    
    if relative_error < 1e-5:
        print("✓ Gradient check PASSED!")
    else:
        print("✗ Gradient check FAILED!")
    
    return relative_error

In [ ]:
# 測試 1: 線性函數
print("=" * 50)
print("Test 1: Linear function f(x) = w^T x")
print("=" * 50)

w = np.array([1.0, 2.0, 3.0])
x = np.array([0.5, -0.3, 0.8])

f_linear = lambda x: linear_function(x, w)
analytic = grad_linear(x, w)

gradient_check(analytic, f_linear, x)

In [ ]:
# 測試 2: L2 範數平方
print("\n" + "=" * 50)
print("Test 2: L2 squared f(x) = ||x||²")
print("=" * 50)

x = np.array([1.0, 2.0, 3.0])
analytic = grad_l2_squared(x)

gradient_check(analytic, l2_squared, x)

In [ ]:
# 測試 3: 二次型
print("\n" + "=" * 50)
print("Test 3: Quadratic form f(x) = x^T A x")
print("=" * 50)

A = np.array([[2.0, 1.0],
              [1.0, 3.0]])
x = np.array([1.0, 2.0])

f_quad = lambda x: quadratic_form(x, A)
analytic = grad_quadratic_form(x, A)

gradient_check(analytic, f_quad, x)

---
## Part 4: 鏈式法則 (Chain Rule)

這是反向傳播的數學基礎！

### 一維版本

如果 $y = f(x)$ 且 $z = g(y)$，則：

$$\frac{dz}{dx} = \frac{dz}{dy} \cdot \frac{dy}{dx}$$

### 多維版本

如果 $\mathbf{y} = f(\mathbf{x})$ 且 $L = g(\mathbf{y})$（$L$ 是純量），則：

$$\frac{\partial L}{\partial x_i} = \sum_j \frac{\partial L}{\partial y_j} \cdot \frac{\partial y_j}{\partial x_i}$$

用矩陣表示：

$$\nabla_\mathbf{x} L = J^T \nabla_\mathbf{y} L$$

其中 $J$ 是 Jacobian 矩陣。

### 為什麼這很重要？

神經網路的反向傳播就是不斷應用鏈式法則：

```
x → [Layer 1] → h1 → [Layer 2] → h2 → ... → [Loss] → L
```

計算 $\frac{\partial L}{\partial \text{Layer 1 參數}}$ 需要用鏈式法則把梯度從 L 傳回去。

### 練習 2: 鏈式法則

考慮複合函數：

$$L = ||\mathbf{y}||^2$$
$$\mathbf{y} = W\mathbf{x} + \mathbf{b}$$

求 $\nabla_\mathbf{x} L$、$\nabla_W L$、$\nabla_\mathbf{b} L$

In [ ]:
def forward(x, W, b):
    """
    Forward pass: y = Wx + b, L = ||y||²
    
    Returns
    -------
    L : float
        Loss value
    cache : dict
        保存的中間值，backward 會用到
    """
    y = W @ x + b
    L = np.sum(y ** 2)
    cache = {'x': x, 'W': W, 'b': b, 'y': y}
    return L, cache


def backward(cache):
    """
    Backward pass: 計算梯度
    
    用鏈式法則：
    ∂L/∂y = 2y (因為 L = ||y||²)
    ∂L/∂W = ∂L/∂y · ∂y/∂W = 2y · x^T
    ∂L/∂b = ∂L/∂y · ∂y/∂b = 2y
    ∂L/∂x = ∂L/∂y · ∂y/∂x = W^T · 2y
    
    Returns
    -------
    grad_x : np.ndarray
    grad_W : np.ndarray
    grad_b : np.ndarray
    """
    x, W, b, y = cache['x'], cache['W'], cache['b'], cache['y']
    
    # 解答：
    # Step 1: ∂L/∂y
    grad_y = 2 * y
    
    # Step 2: 用鏈式法則
    grad_x = W.T @ grad_y
    grad_W = np.outer(grad_y, x)  # grad_y @ x.T
    grad_b = grad_y
    
    return grad_x, grad_W, grad_b

In [ ]:
# 測試鏈式法則
np.random.seed(42)

# 隨機初始化
x = np.random.randn(3)
W = np.random.randn(2, 3)
b = np.random.randn(2)

# Forward
L, cache = forward(x, W, b)
print(f"Loss L = {L:.4f}")

# Backward
grad_x, grad_W, grad_b = backward(cache)

# 驗證 grad_x
print("\n" + "=" * 50)
print("Checking ∂L/∂x")
print("=" * 50)
f_x = lambda x: forward(x, W, b)[0]
gradient_check(grad_x, f_x, x.copy())

# 驗證 grad_b
print("\n" + "=" * 50)
print("Checking ∂L/∂b")
print("=" * 50)
f_b = lambda b: forward(x, W, b)[0]
gradient_check(grad_b, f_b, b.copy())

# 驗證 grad_W (需要 flatten)
print("\n" + "=" * 50)
print("Checking ∂L/∂W")
print("=" * 50)
def f_W(W_flat):
    W_mat = W_flat.reshape(W.shape)
    return forward(x, W_mat, b)[0]

gradient_check(grad_W.flatten(), f_W, W.flatten().copy())

---
## Part 5: 梯度下降預熱

現在你知道梯度是什麼了，來看看梯度下降如何找函數的最小值。

### 梯度下降算法

$$\mathbf{x}_{t+1} = \mathbf{x}_t - \eta \nabla f(\mathbf{x}_t)$$

其中 $\eta$ 是學習率 (learning rate)。

**直覺**：往「下坡」方向走一小步。

In [ ]:
# 視覺化梯度下降
# 最小化 f(x, y) = x² + 2y²（橢圓形的碗）

def f(xy):
    x, y = xy
    return x**2 + 2*y**2

def grad_f(xy):
    x, y = xy
    return np.array([2*x, 4*y])

# 梯度下降
def gradient_descent(f, grad_f, x0, lr=0.1, n_iter=20):
    x = x0.copy()
    history = [x.copy()]
    
    for _ in range(n_iter):
        g = grad_f(x)
        x = x - lr * g
        history.append(x.copy())
    
    return np.array(history)

# 從 (3, 3) 開始
x0 = np.array([3.0, 3.0])
history = gradient_descent(f, grad_f, x0, lr=0.15, n_iter=15)

# 畫圖
x = np.linspace(-4, 4, 100)
y = np.linspace(-4, 4, 100)
X, Y = np.meshgrid(x, y)
Z = X**2 + 2*Y**2

plt.figure(figsize=(10, 8))
plt.contour(X, Y, Z, levels=20, cmap='viridis')
plt.colorbar(label='f(x, y)')

# 畫軌跡
plt.plot(history[:, 0], history[:, 1], 'ro-', markersize=8, linewidth=2, label='GD path')
plt.plot(history[0, 0], history[0, 1], 'go', markersize=15, label='Start')
plt.plot(0, 0, 'b*', markersize=20, label='Minimum')

plt.xlabel('x')
plt.ylabel('y')
plt.title('梯度下降尋找 f(x,y) = x² + 2y² 的最小值')
plt.legend()
plt.axis('equal')
plt.grid(True, alpha=0.3)
plt.show()

print(f"起點: {history[0]}")
print(f"終點: {history[-1]}")
print(f"終點的 f 值: {f(history[-1]):.6f}")

---
## Summary

這個 notebook 你學到了：

1. **梯度**是多維導數，指向函數增長最快的方向
2. **常見函數的梯度**：
   - $f = \mathbf{w}^T\mathbf{x}$ → $\nabla f = \mathbf{w}$
   - $f = ||\mathbf{x}||^2$ → $\nabla f = 2\mathbf{x}$
   - $f = \mathbf{x}^T A \mathbf{x}$ → $\nabla f = (A + A^T)\mathbf{x}$
3. **數值微分**可以用來驗證你手推的梯度
4. **鏈式法則**是反向傳播的基礎
5. **梯度下降**往 $-\nabla f$ 方向走來找最小值

---

**下一個 notebook**: `03_probability.ipynb` - 機率基礎